In [5]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd


# Import Dataset, Processing

In [6]:
DF = pd.read_csv('gurobiTime.csv',index_col=0)
DF = DF.drop(['player.1'],axis=1)


positionDF = []
num_positions = len(DF['position'].unique())


for i in DF['position'].unique():
    positionDF.append( (DF[DF['position'] == i]) )

    

Make Gurobi datastructures to help represent the problem

We create dictionaries for every single variable for each player, to allow for efficient access and ease of use.  The keys are Players, the tupleList objects are returnPlayer, playerVariance, playerCost , and position


QB, RB, RB, WR, WR, WR, TE, D, K 



In [82]:
bigDict = {}
for df_iter in range(len(positionDF)):
    
    currPosition = positionDF[df_iter]  
    for player in currPosition.index:
        returnPlayer = currPosition.loc[player]['mean']
        playerVariance = currPosition.loc[player]['variance']
        playerCost = currPosition.loc[player]['cost']
        position = currPosition.loc[player]['position']
        bigDict[player] = [returnPlayer, playerVariance, playerCost , position]
         
player, returnPlayer, playerVariance, playerCost , position = gp.multidict(bigDict)



playerDecisionVars = gp.tuplelist( [ (choice, returnPlayer.select(choice)[0], playerVariance.select(choice)[0])  for choice in player ] )



positionConstraints = gp.tupledict( { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1,'FLEX':1})#'K':0} )




In [33]:
def printToolBox():
    print('\n, player: list of names of players. Essentially, indexes')
    print('\n returnPlayer, playerVariance, playerCost , position : all tuple dictionaries, indexed by player ')
    print('\n positionConstraints: tupledict for constraint amounts, index by position')
    print('\n playerDecisionVars: tuplelist, essentially [player,E[return],Var[Return]]')
    print('\n players: gurobi tupledict; Binary decision variables: O if player is not chosen, 1 if chosen. Indexed by player  ')
    print('\n flexPossible : gurobi TupleDict: contains all player objects that are possible for flex position')
    print('\n flexPlayers: gurobi decision variables, tupleDict, 0,1 for binary flex players')
    
    #returns true if a player is valid for a flex position
def flexValid(player):
    if position[player] == "WR" or position[player] == 'RB' or position[player] == 'TE':
        return True
    else:
        return False
    


list

In [196]:
try:
    m = gp.Model('QMIP')
    #add binary decision variables on whether 
    

    
    #add decision variables
    players = m.addVars( playerDecisionVars, vtype=GRB.BINARY, name='players') 
        # get easy to use Hashmap for players
    m.update()
    flexPossible =gp.tuplelist([(plyr,players.select(plyr)[0]) for plyr in player if flexValid(plyr)])
    x = {}
    for i in flexPossible:
        key = i[0]
        x[key] = i[1]
    flexPossible = gp.tupledict(x)
    #add flex variables as a another type of decision variables
    
    flexPlayers = m.addVars(flexPossible, vtype=GRB.BINARY,name='flexPlayers')
    
    m.update()

    #add position constraint. We set upper bound to be one and keep the constraint a continuous variable since the 
    #model will keep integer solutions
    
    #positionConstratint = m.addVars(positionConstraints, ub =1 , name='positionConstr')
    

    # set objective funcition
    m.setObjective(  - gp.quicksum( returnPlayer[choice] *players[choice,retrn,var] for choice, retrn, var in players )
                   - gp.quicksum(returnPlayer[choice] * flexPlayers[choice] for choice in flexPossible)
                   + gp.quicksum( playerVariance[choice] *players[choice,retrn,var] for choice, retrn, var in players )
                    + gp.quicksum( playerVariance[choice] *flexPlayers[choice] for choice in flexPossible ),GRB.MINIMIZE)
    
    #constraint: sum of each position must be a constraint:
    #for pos in DF['position'].unique():
        
        
    posConstrs = {} 
    for pos in DF['position'].unique():
        if 'pos' == 'FLEX':
            continue
        players_in_pos = [playr for playr in player if position[playr] == pos]
        
        posConstrs[pos] = m.addConstr( gp.quicksum( [ players.select(playr)[0] for playr in player if position[playr] == pos ] ) == positionConstraints[pos] )
        
    
    #salary constraint
    costConstr = m.addConstr(gp.quicksum( [ playerCost[playr] *players.select(playr)[0] for playr in player ]) <= 60000 )
    
    flexIndex = [player.select(plyr)[0] for plyr in player if flexValid(plyr)]

    flxConstrs = {}
    for plyr in flexIndex:
       # print(flexPlayers.select(plyr)[0])
        flxConstrs[plyr] = m.addConstr( (flexPlayers.select(plyr)[0] + players.select(plyr)[0] <= 1  )  )
  
    #flexConstrs = m.addConstr( (flexPlayers.select(plyr)[0] + players.select(plyr)[0] for plyr in player if flexValid(plyr)  <= positionConstraints["FLEX"] ) )
    
    
    
    #flex constraints
  #  flexConstr = m.addConstr(gp.quicksum([ players.prod(players.select(playr),'*') for playr in player if playr == 'WR' ] ) == 1 )
    
    m.update()
    
    m.optimize()
    
    foundPlayers = m.X
    
    
    teamRoster = [ player[i] for i in range(len(player)) if foundPlayers[i]==1 ]
    print('\noptimal team roster: ',teamRoster)
    
    sum = 0
    for i in teamRoster:
        sum += playerCost[i] 
    sum 


    constr = { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1} 

    for pos in DF['position'].unique():
        for i in teamRoster:
            if position[i] == pos:
                constr[pos] -= 1
    for key,value in constr.items():
        if value != 0:
            print('failure to meet pos constraint:',key)
    print('valid roster, cost =',sum)    
    
    print('\n expected return - risk :',m.objVal)

except gp.GurobiError as e:
    print('Error code ' + str(e.errno) + ": " + str(e))

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (linux64)
Optimize a model with 409 rows, 881 columns and 1762 nonzeros
Model fingerprint: 0x3c232bf3
Variable types: 0 continuous, 881 integer (881 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+03]
  Objective range  [4e-14, 4e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+04]
Found heuristic solution: objective 349.6047467
Presolve removed 409 rows and 881 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds
Thread count was 1 (of 4 available processors)

Solution count 2: -20.4581 
No other solutions better than -20.4581

Optimal solution found (tolerance 1.00e-04)
Best objective -2.045807333333e+01, best bound -2.045807333333e+01, gap 0.0000%

optimal team roster:  ['Carolina', 'chrishogan/2530515', 'dontemoncrief/2543614', 'zayjones/2557863', 'bilalpowell/2495328', 'devontaebooker/2555223', 't.j.logan/2558273', 'bakermayfield/

In [166]:
flexTest = [player for plyr in player if flexValid(plyr)][0]
l = []
r = []
for plyr in flexTest:
    l.append(players.select(plyr)[0])
    r.append(flexPlayers.select(plyr))
#flexPlayers[('a.j.brown/2562238,<gurobi.Var players[a.j.brown/2562238,7.619999999999999,83.35200000000006]>')]

In [165]:
(flexPlayers.select(plyr)[0] + players.select(plyr)[0] for plyr in player if flexValid(plyr)  <= 1 ) 


<generator object <genexpr> at 0x7fe45f8f2ad0>

In [ ]:
gp.quicksum([ players.prod(players.select(playr),'*')[ for playr in player if playr == 'WR' ] ) == 1 

In [ ]:
gp.quicksum( [   playerCost[playr] *players.select(playr)[0] for playr in player ]) <= 50000

In [20]:
teamRoster

sum = 0
for i in teamRoster:
    sum += playerCost[i] 
sum 


constr = { 'QB':1, "RB":3,"WR":3, 'TE':1,"DEF":1} 

for pos in DF['position'].unique():
    for i in teamRoster:
        if position[i] == pos:
            constr[pos] -= 1
for key,value in constr.items():
    if value != 0:
        print('failure to meet pos constraint:',key)
print('valid roster, cost =',sum)


valid roster, cost = 50600.0


In [171]:
for plyr in flexIndex:
        print(flexPlayers[plyr])
        #flxConstrs[plyr] =(flexPlayers.select(plyr) + players.select(plyr)[0] <= 1  )  
  

KeyError: 'Arizona'

In [197]:
flexIndex = [player.select(plyr)[0] for plyr in player if flexValid(plyr)]


9